In [ ]:
%matplotlib inline

# Calibration of a polynomial regression.


In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation

from gemseo.algos.design_space import DesignSpace
from gemseo.mlearning.core.calibration import MLAlgoCalibration
from gemseo.mlearning.regression.quality.mse_measure import MSEMeasure
from gemseo.problems.dataset.rosenbrock import create_rosenbrock_dataset

## Load the dataset




In [ ]:
dataset = create_rosenbrock_dataset(opt_naming=False, n_samples=25)

## Define the measure




In [ ]:
test_dataset = create_rosenbrock_dataset(opt_naming=False)
measure_evaluation_method_name = "TEST"
measure_options = {"test_data": test_dataset}

## Calibrate the degree of the polynomial regression

### Define and execute the calibration




In [ ]:
calibration_space = DesignSpace()
calibration_space.add_variable("degree", 1, "integer", 1, 10, 1)
calibration = MLAlgoCalibration(
    "PolynomialRegressor",
    dataset,
    ["degree"],
    calibration_space,
    MSEMeasure,
    measure_evaluation_method_name=measure_evaluation_method_name,
    measure_options=measure_options,
)
calibration.execute(algo_name="PYDOE_FULLFACT", n_samples=10)
x_opt = calibration.optimal_parameters
f_opt = calibration.optimal_criterion
degree = x_opt["degree"][0]
f"optimal degree = {degree}; optimal criterion = {f_opt}"

### Get the history




In [ ]:
calibration.dataset

### Visualize the results




In [ ]:
degree = calibration.get_history("degree")
criterion = calibration.get_history("criterion")
learning = calibration.get_history("learning")

plt.plot(degree, criterion, "-o", label="test", color="red")
plt.plot(degree, learning, "-o", label="learning", color="blue")
plt.xlabel("polynomial degree")
plt.ylabel("quality")
plt.axvline(x_opt["degree"], color="red", ls="--")
plt.legend()
plt.show()

## Calibrate the ridge penalty of the polynomial regression

### Define and execute the calibration




In [ ]:
calibration_space = DesignSpace()
calibration_space.add_variable("penalty_level", 1, "float", 0.0, 100.0, 0.0)
calibration = MLAlgoCalibration(
    "PolynomialRegressor",
    dataset,
    ["penalty_level"],
    calibration_space,
    MSEMeasure,
    measure_evaluation_method_name=measure_evaluation_method_name,
    measure_options=measure_options,
    degree=10,
)
calibration.execute(algo_name="PYDOE_FULLFACT", n_samples=10)
x_opt = calibration.optimal_parameters
f_opt = calibration.optimal_criterion
x_opt["penalty_level"][0], f_opt

### Get the history




In [ ]:
calibration.dataset

### Visualize the results




In [ ]:
penalty_level = calibration.get_history("penalty_level")
criterion = calibration.get_history("criterion")
learning = calibration.get_history("learning")

plt.plot(penalty_level, criterion, "-o", label="test", color="red")
plt.plot(penalty_level, learning, "-o", label="learning", color="blue")
plt.axvline(x_opt["penalty_level"], color="red", ls="--")
plt.xlabel("ridge penalty")
plt.ylabel("quality")
plt.legend()
plt.show()

## Calibrate the lasso penalty of the polynomial regression

### Define and execute the calibration




In [ ]:
calibration_space = DesignSpace()
calibration_space.add_variable("penalty_level", 1, "float", 0.0, 100.0, 0.0)
calibration = MLAlgoCalibration(
    "PolynomialRegressor",
    dataset,
    ["penalty_level"],
    calibration_space,
    MSEMeasure,
    measure_evaluation_method_name=measure_evaluation_method_name,
    measure_options=measure_options,
    degree=10,
    l2_penalty_ratio=0.0,
)
calibration.execute(algo_name="PYDOE_FULLFACT", n_samples=10)
x_opt = calibration.optimal_parameters
f_opt = calibration.optimal_criterion
x_opt["penalty_level"][0], f_opt

### Get the history




In [ ]:
calibration.dataset

### Visualize the results




In [ ]:
penalty_level = calibration.get_history("penalty_level")
criterion = calibration.get_history("criterion")
learning = calibration.get_history("learning")

plt.plot(penalty_level, criterion, "-o", label="test", color="red")
plt.plot(penalty_level, learning, "-o", label="learning", color="blue")
plt.axvline(x_opt["penalty_level"], color="red", ls="--")
plt.xlabel("lasso penalty")
plt.ylabel("quality")
plt.legend()
plt.show()

## Calibrate the elasticnet penalty of the polynomial regression

### Define and execute the calibration




In [ ]:
calibration_space = DesignSpace()
calibration_space.add_variable("penalty_level", 1, "float", 0.0, 40.0, 0.0)
calibration_space.add_variable("l2_penalty_ratio", 1, "float", 0.0, 1.0, 0.5)
calibration = MLAlgoCalibration(
    "PolynomialRegressor",
    dataset,
    ["penalty_level", "l2_penalty_ratio"],
    calibration_space,
    MSEMeasure,
    measure_evaluation_method_name=measure_evaluation_method_name,
    measure_options=measure_options,
    degree=10,
)
calibration.execute(algo_name="PYDOE_FULLFACT", n_samples=100)
x_opt = calibration.optimal_parameters
f_opt = calibration.optimal_criterion
x_opt["penalty_level"][0], x_opt["l2_penalty_ratio"][0], f_opt

### Get the history




In [ ]:
calibration.dataset

### Visualize the results




In [ ]:
penalty_level = calibration.get_history("penalty_level").flatten()
l2_penalty_ratio = calibration.get_history("l2_penalty_ratio").flatten()
criterion = calibration.get_history("criterion").flatten()
learning = calibration.get_history("learning").flatten()

triang = Triangulation(penalty_level, l2_penalty_ratio)

fig = plt.figure()
ax = fig.add_subplot(1, 2, 1)
ax.tricontourf(triang, criterion, cmap="Purples")
ax.scatter(x_opt["penalty_level"][0], x_opt["l2_penalty_ratio"][0])
ax.set_xlabel("penalty level")
ax.set_ylabel("l2 penalty ratio")
ax.set_title("Test measure")
ax = fig.add_subplot(1, 2, 2)
ax.tricontourf(triang, learning, cmap="Purples")
ax.scatter(x_opt["penalty_level"][0], x_opt["l2_penalty_ratio"][0])
ax.set_xlabel("penalty level")
ax.set_ylabel("l2 penalty ratio")
ax.set_title("Learning measure")

plt.show()

### Add an optimization stage




In [ ]:
calibration_space = DesignSpace()
calibration_space.add_variable("penalty_level", 1, "float", 0.0, 40.0, 0.0)
calibration_space.add_variable("l2_penalty_ratio", 1, "float", 0.0, 1.0, 0.5)
calibration = MLAlgoCalibration(
    "PolynomialRegressor",
    dataset,
    ["penalty_level", "l2_penalty_ratio"],
    calibration_space,
    MSEMeasure,
    measure_evaluation_method_name=measure_evaluation_method_name,
    measure_options=measure_options,
    degree=10,
)
calibration.execute("NLOPT_COBYLA", max_iter=100)
x_opt2 = calibration.optimal_parameters
f_opt2 = calibration.optimal_criterion

fig = plt.figure()
ax = fig.add_subplot(1, 2, 1)
ax.tricontourf(triang, criterion, cmap="Purples")
ax.scatter(x_opt["penalty_level"][0], x_opt["l2_penalty_ratio"][0])
ax.scatter(x_opt2["penalty_level"][0], x_opt2["l2_penalty_ratio"][0], color="red")
ax.set_xlabel("penalty level")
ax.set_ylabel("l2 penalty ratio")
ax.set_title("Test measure")
ax = fig.add_subplot(1, 2, 2)
ax.tricontourf(triang, learning, cmap="Purples")
ax.scatter(x_opt["penalty_level"][0], x_opt["l2_penalty_ratio"][0])
ax.scatter(x_opt2["penalty_level"][0], x_opt2["l2_penalty_ratio"][0], color="red")
ax.set_xlabel("penalty level")
ax.set_ylabel("l2 penalty ratio")
ax.set_title("Learning measure")
plt.show()

n_iterations = len(calibration.scenario.disciplines[0].cache)